In [5]:
import logging
from collections import defaultdict
from gensim import corpora
import pprint
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

print(corpus)

2020-06-11 14:56:05,369 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-06-11 14:56:05,370 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [6]:
#Model
from gensim import models

lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
doc = 'Human Computer Interaction'
vec_bow = dictionary.doc2bow(doc.lower().split(' '))
vec_lsi = lsi[vec_bow]
pprint.pprint(vec_lsi)

2020-06-11 15:18:01,659 : INFO : using serial LSI version on this node
2020-06-11 15:18:01,660 : INFO : updating model with new documents
2020-06-11 15:18:01,661 : INFO : preparing a new chunk of documents
2020-06-11 15:18:01,662 : INFO : using 100 extra samples and 2 power iterations
2020-06-11 15:18:01,662 : INFO : 1st phase: constructing (12, 102) action matrix
2020-06-11 15:18:01,671 : INFO : orthonormalizing (12, 102) action matrix
2020-06-11 15:18:01,708 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2020-06-11 15:18:01,710 : INFO : computing the final decomposition
2020-06-11 15:18:01,711 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2020-06-11 15:18:01,712 : INFO : processed documents up to #9
2020-06-11 15:18:01,713 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"response" + 0.265*"time" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2020-06-11 15:18:01,715 : INFO : topic #1(2

[(0, 0.46182100453271563), (1, 0.07002766527900073)]


In [9]:
from gensim import similarities

index = similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]
print(list(enumerate(sims)))

2020-06-11 15:23:41,991 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2020-06-11 15:23:41,993 : INFO : creating matrix with 9 documents and 2 features


[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.09879464), (8, 0.050041765)]


In [10]:
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in enumerate(sims_sorted):
    print(s, documents[i])

(2, 0.9984453) Human machine interface for lab abc computer applications
(0, 0.998093) A survey of user opinion of computer system response time
(3, 0.9865886) The EPS user interface management system
(1, 0.93748635) System and human system engineering testing of EPS
(4, 0.90755945) Relation of user perceived response time to error measurement
(8, 0.050041765) The generation of random binary unordered trees
(7, -0.09879464) The intersection graph of paths in trees
(6, -0.10639259) Graph minors IV Widths of trees and well quasi ordering
(5, -0.12416792) Graph minors A survey
